In [6]:
import gym

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import envs.custom_safety_envs    

from garage.envs import GymEnv
from wrappers import DiversityWrapper, SafetyEnvStateAppender, Renderer

In [19]:
env = gym.make('Safexp-PointGoal0-v0')
env1 = DiversityWrapper(env, number_skills=20)
env2 = SafetyEnvStateAppender(env1)
env3 = Renderer(env2, os.path.join('.', 'videos'))
env4 = GymEnv(env3, max_episode_length=1000)

In [11]:
env.metadata

{'render.modes': []}

In [13]:
env.set_skill(2)

In [15]:
env.metadata

{'render.modes': [], 'skill': 2, 'ep_name': 'skill_2'}

In [36]:
env4._env.metadata.get('ep_name')

'skill_2'

In [37]:
env.reset()

array([ 0.        ,  0.        ,  9.81      ,  0.        ,  0.        ,
        0.        ,  0.51500891,  0.59559477,  0.08058586,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.11540385, -0.48649969,  0.        ,
        0.        ,  0.        ,  0.        ])

In [38]:
env.metadata

{'render.modes': [], 'skill': 2, 'ep_name': 'skill_2'}

In [41]:
env4._env.env.metadata['ep_name'] = 'pooey'

In [42]:
env4.metadata

{'render.modes': [], 'skill': 2, 'ep_name': 'pooey'}

In [43]:
import copy

In [44]:
copied_env = copy.deepcopy(env4)

In [48]:
copied_env.metadata

{'render.modes': [], 'skill': 2, 'ep_name': 'wowser'}

In [47]:
copied_env._env.env.metadata['ep_name'] = 'wowser'

In [49]:
copied_env._env.metadata

{'render.modes': [], 'skill': 2, 'ep_name': 'wowser'}

In [50]:
id(env4.metadata)

47353639328448

In [51]:
id(env4._env.env.metadata)

47353639328448

In [52]:
id(env4._env.env.env.metadata)

47353639328448

In [54]:
type(env4._env.env.env.env)

safety_gym.envs.engine.Engine

In [56]:
id(env4._env.env.env.env.metadata)

47353639328448

In [70]:
copied_env = copy.deepcopy(env4)

In [72]:
print(id(copied_env.metadata))
print(id(copied_env._env.metadata))
print(id(copied_env._env.env.metadata))
print(id(copied_env._env.env.env.metadata))
print(id(copied_env._env.env.env.env.metadata))

47359527291136
47359527291136
47359527291136
47359527291136
47353639328448


In [71]:
print(id(env4.metadata))
print(id(env4._env.metadata))
print(id(env4._env.env.metadata))
print(id(env4._env.env.env.metadata))
print(id(env4._env.env.env.env.metadata))

47353639328448
47353639328448
47353639328448
47353639328448
47353639328448


In [69]:
type(env4._env.env.env)

wrappers.diversity_wrapper.DiversityWrapper